<a href="https://colab.research.google.com/github/LotusBro98/Lanat2019/blob/master/Machine_Learning_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Машинное обучение
## Генеративные состязательные сети

---

<img width="100%" src="https://pa1.narvii.com/6910/9641f0ed17e10ad698e4e9c23a3de25550339fc5r1-811-397_hq.gif">

---

В данном примере будет рассмотрена архитектура DCGAN для генерации Pixel Art персонажей

In [0]:
!pip install tensorflow-gpu==2.0.0-beta1
import tensorflow as tf

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

import os
import glob
import random
import time
import sys

import tensorflow.keras.layers as layers

from IPython import display

In [0]:
SIZE = 64
MAX_IMAGES = 1000
OUTPUT_CHANNELS = 4
N_INPUT_PARAMS = 512
BATCH_SIZE = 1

## Загрузим и подготовим датасет

Поместите изображения на свой Google Диск в папку "Мой Диск/pixelartgen/images"

In [0]:
from google.colab import drive
drive.mount('/content/drive')

app_dir = "/content/drive/My Drive/pixelartgen"
!mkdir -p "{app_dir}"
os.chdir(app_dir)

img_dir = "images"
!mkdir -p "{img_dir}"

In [0]:
def load_image(filename):
  image = tf.io.read_file(filename)
  image = tf.io.decode_image(image, channels=4)

  h, w = image.shape[:2]
  if w > h:
    offset_h = (w - h) // 2
    offset_w = 0
  else:
    offset_h = 0
    offset_w = (h - w) // 2

  image = tf.cast(image, tf.float32)
  image = image / 255.0

  image = tf.image.pad_to_bounding_box(image, offset_h, offset_w, max(w, h), max(w, h))
  alpha_1 = tf.ones_like(image)
  alpha_1 = tf.transpose(alpha_1, (2, 0, 1))

  image = tf.transpose(image, (2, 0, 1))
  image = image * image[-1] + alpha_1 * (1.0 - image[-1])
  image = tf.transpose(image, (1, 2, 0))

  image = tf.image.resize(image, (SIZE, SIZE), method='nearest')
  image = image * 2 - 1

  return image

In [0]:
files = glob.glob(os.path.join(img_dir, "*.*"))
files = files[:MAX_IMAGES]
train_images = list(map(load_image, files))
train_images = tf.stack(train_images)

train_dataset = tf.data.Dataset.from_tensor_slices(train_images)
train_dataset = train_dataset.batch(BATCH_SIZE)

print(train_dataset)

### Отобразим несколько картинок из датасета

In [0]:
N_PLOTS = 4
plt.figure(figsize=(10, 10))
for i in range(N_PLOTS * N_PLOTS):
  plt.subplot(N_PLOTS, N_PLOTS, i + 1)
  plt.imshow(random.choice(train_images) * 0.5 + 0.5)
  plt.axis('off')
plt.show()

## Определим модель генератора

In [0]:
def upsample(x, filters, size=5):
    x = layers.Conv2DTranspose(filters, (size, size), strides=(2, 2), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    return x

def make_generator_model():
    inputs = layers.Input(shape=(N_INPUT_PARAMS,))
    x = inputs

    x = layers.Dense(512, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Reshape((1, 1, 512))(x)

    x = upsample(x, 256)     # 2 x 2 x 256
    x = upsample(x, 128)     # 4 x 4 x 128
    x = upsample(x, 64)      # 8 x 8 x 64
    x = upsample(x, 32)      # 16 x 16 x 32
    x = upsample(x, 16)      # 32 x 32 x 16
    x = upsample(x, 8)       # 64 x 64 x 8

    x = layers.Conv2DTranspose(OUTPUT_CHANNELS, (5, 5), padding='same', use_bias=False, activation='tanh')(x)
                             # 64 x 64 x 4
                             
    assert tuple(x.shape) == (None, SIZE, SIZE, OUTPUT_CHANNELS)

    return tf.keras.Model(inputs=inputs, outputs=x)

In [0]:
def downsample(x, filters, size=5):
    x = layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same')(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.3)(x)
    return x

def make_discriminator_model():
    inputs = layers.Input(shape=(SIZE, SIZE, OUTPUT_CHANNELS))
    x = inputs

    x = downsample(x, 16)     # 32 x 32 x 16
    x = downsample(x, 32)     # 16 x 16 x 32
    x = downsample(x, 64)     # 8 x 8 x 64
    x = downsample(x, 128)    # 4 x 4 x 128 
    x = downsample(x, 256)    # 2 x 2 x 256
    x = downsample(x, 512)    # 1 x 1 x 512  

    x = layers.Flatten()(x)
    x = layers.Dense(1)(x)

    return tf.keras.Model(inputs=inputs, outputs=x)

## Определим лосс-функции

In [0]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [0]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [0]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [0]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

## Приступим к обучению

In [0]:
# We will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([N_PLOTS * N_PLOTS, N_INPUT_PARAMS])

In [0]:
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, N_INPUT_PARAMS])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [0]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_step(image_batch)

    if (epoch + 1) % 200 == 0:
      # Produce images for the GIF as we go
      display.clear_output(wait=True)
      generate_and_save_images(seed, epoch)

    # Save the model every 15 epochs
    if (epoch + 1) % 10000 == 0:
      generator.save("generator.h5")
      discriminator.save("discriminator.h5")

    sys.stdout.write('\rTime for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
    sys.stdout.flush()

In [0]:
!mkdir -p gen

def generate_and_save_images(test_input, epoch):
  generated = generator(test_input)

  plt.figure(figsize=(10, 10))
  for i in range(N_PLOTS * N_PLOTS):
    plt.subplot(N_PLOTS, N_PLOTS, i + 1)
    plt.imshow(generated[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()
  plt.savefig('gen/image_at_epoch_{:04d}.png'.format(epoch))

## Обучаем!

In [0]:
generator = make_generator_model()
discriminator = make_discriminator_model()

In [0]:
# Восстанавливаем сохраненную модель, если она есть
try:
  generator.load_weights("generator.h5")
  discriminator.load_weights("discriminator.h5")
  print("Restored")
except:
  print("Failed to restore. Starting all over")

### И вот теперь наконец обучаем!!!

In [0]:
EPOCHS = 10000
train(train_dataset, EPOCHS)